In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld

URM_all, _, _, _, _ = ld.getCOOs()
ICM_all = ld.getICMall()

SEED = 6180

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.7, seed=SEED)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13649 ( 0.0%) Users that have less than 1 test interactions


In [4]:
import os

ofp = "../models_temp/Similarity_Hybrid/Split07/"


models_to_combine_best = {
                            'IALS' : {"num_factors": 29, "epochs": 50, "confidence_scaling": "log", "alpha": 0.001, "epsilon": 0.001, "reg": 0.01},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'ICM_all': {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'TopPop': {},
                            'PureSVD': {'num_factors': 30},
                            'UserKNN': {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'EASE_R': {'l2_norm': 3750},
}

In [6]:
def model_init(recommender, name, args):
    path_name = ofp + name + str(SEED) + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name + str(SEED))
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name + str(SEED))

In [7]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
# from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

In [8]:
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender_FAST
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

In [9]:
recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender2 = SLIM_BPR_Cython(URM_train)
recommender3 = RP3betaRecommender(URM_train)
recommender4 = ItemKNNCBFRecommender(URM_train, ICM_all)
recommender5 = IALSRecommender(URM_train)
recommender6 = P3alphaRecommender(URM_train)
recommender7 = UserKNNCFRecommender(URM_train)
recommender8 = PureSVDRecommender(URM_train)
recommender9 = TopPop(URM_train)
recommender10 = EASE_R_Recommender(URM_train)

model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
model_init(recommender2, 'SLIMBPR', models_to_combine_best['SLIMBPR'])
model_init(recommender3, 'RP3beta', models_to_combine_best['RP3beta'])
model_init(recommender4, 'ICM_all', models_to_combine_best['ICM_all'])
model_init(recommender5, 'IALS', models_to_combine_best['IALS'])
model_init(recommender6, 'P3alpha', models_to_combine_best['P3alpha'])
model_init(recommender7, 'UserKNN', models_to_combine_best['UserKNN'])
model_init(recommender8, 'PureSVD', models_to_combine_best['PureSVD'])
model_init(recommender9, 'TopPop', models_to_combine_best['TopPop'])
model_init(recommender10, 'EASE_R', models_to_combine_best['EASE_R'])

../models_temp/Similarity_Hybrid/Split07/SLIMER6180.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/Split07/SLIMER6180'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/Split07/SLIMBPR6180.zip
Model found!
SLIM_BPR_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/Split07/SLIMBPR6180'
SLIM_BPR_Recommender: Loading complete
../models_temp/Similarity_Hybrid/Split07/RP3beta6180.zip
Model found!
RP3betaRecommender: Loading model from file '../models_temp/Similarity_Hybrid/Split07/RP3beta6180'
RP3betaRecommender: Loading complete
../models_temp/Similarity_Hybrid/Split07/ICM_all6180.zip
Model found!
ItemKNNCBFRecommender: Loading model from file '../models_temp/Similarity_Hybrid/Split07/ICM_all6180'
ItemKNNCBFRecommender: Loading complete
../models_temp/Similarity_Hybrid/Split07/IALS6180.zip
Model found!
IALSRecommender: Loading model from file '../models_temp/Similarity_Hybrid/Split07/IALS

In [ ]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [ ]:
rank_models(evaluator_validation, [recommender1, recommender2])

In [10]:
from Recommenders.HybridScores.DifferentStructure import ThreeDifferentModelRecommender
import optuna

class Objective(object):
    def __init__(self, URM_train, recommender1, recommender2, recommender3, evaluator):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train = URM_train
        self.evaluator = evaluator
        self.recommender_1 = recommender1
        self.recommender_2 = recommender2
        self.recommender_3 = recommender3

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        w_1 = trial.suggest_uniform('w_1', 0, 1)
        w_2 = trial.suggest_uniform('w_2', 0, 1)
        w_3 = trial.suggest_uniform('w_3', 0, 1)

        recommender_final = ThreeDifferentModelRecommender(self.URM_train.tocsr(), self.recommender_1, self.recommender_2, self.recommender_3)
        recommender_final.fit(2, w_1, w_2, w_3)
        result_dict, _ = self.evaluator.evaluateRecommender(recommender_final)

        map_v = -result_dict.loc[10]['MAP']
        if map_v >= -0.245: # minimum acceptable map
            # Calculate the penalty.
            trial.report(map_v, 0)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return map_v

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(Objective(URM_train, recommender1, recommender7, recommender10, evaluator_validation), n_trials=500)

print(study.best_params)

[I 2022-01-07 01:31:11,923] A new study created in memory with name: no-name-1923cb75-d084-4905-9bbf-c86b10c1c6fa


CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.3394612264839287
UserKNNCFRecommender with weight beta: 0.34502850487873876
EASE_R_Recommender with weight gamma: 0.3155102686373325
Norm type: 2
EvaluatorHoldout: Processed 13000 (95.2%) in 5.11 min. Users per second: 42
EvaluatorHoldout: Processed 13649 (100.0%) in 5.16 min. Users per second: 44


[I 2022-01-07 01:36:21,715] Trial 0 finished with value: -0.3445475771816546 and parameters: {'w_1': 0.7822926119035999, 'w_2': 0.7951224740995896, 'w_3': 0.7270973321201698}. Best is trial 0 with value: -0.3445475771816546.


CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.49066379831851786
UserKNNCFRecommender with weight beta: 0.11580907623780365
EASE_R_Recommender with weight gamma: 0.39352712544367857
Norm type: 2
EvaluatorHoldout: Processed 13649 (100.0%) in 4.93 min. Users per second: 46


[I 2022-01-07 01:41:17,436] Trial 1 finished with value: -0.34431523269918207 and parameters: {'w_1': 0.9471330353915388, 'w_2': 0.2235473704783031, 'w_3': 0.759629184194312}. Best is trial 0 with value: -0.3445475771816546.


...

CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.4773391052393999
UserKNNCFRecommender with weight beta: 0.2644423874677183
EASE_R_Recommender with weight gamma: 0.25821850729288176
Norm type: 2
EvaluatorHoldout: Processed 13649 (100.0%) in 4.92 min. Users per second: 46


[I 2022-01-07 09:57:50,342] Trial 102 finished with value: -0.3452442326724335 and parameters: {'w_1': 0.8627336386221112, 'w_2': 0.47794815183122713, 'w_3': 0.4666992289362723}. Best is trial 101 with value: -0.3453712871574991.


CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.4466422732099029
UserKNNCFRecommender with weight beta: 0.2940321652478269
EASE_R_Recommender with weight gamma: 0.25932556154227016
Norm type: 2
EvaluatorHoldout: Processed 13649 (100.0%) in 4.91 min. Users per second: 46


[I 2022-01-07 10:02:45,051] Trial 103 finished with value: -0.34519311258339874 and parameters: {'w_1': 0.9216272090173308, 'w_2': 0.6067227848164842, 'w_3': 0.5351071939371354}. Best is trial 101 with value: -0.3453712871574991.


CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.4824673230295258
UserKNNCFRecommender with weight beta: 0.2294839501789725
EASE_R_Recommender with weight gamma: 0.28804872679150173
Norm type: 2


In [ ]:
import numpy as np

# TWO RECOMMENDERS

def test_percentage(recommender_1, recommender_2, evaluator, step):
    recommender = ItemKNNScoresHybridTwoRecommender_PRUNE(URM_train, recommender_1, recommender_2)
    results = []
    alp_space = np.linspace(0.5, 0.55, step, True)
    for alp in alp_space:
        recommender.fit(alpha = alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [ ]:
alp_space, results = test_percentage(recommender1, recommender2, evaluator_validation, 2)

In [ ]:
import matplotlib.pyplot as plt

_ = plt.figure(figsize=(16, 9))
plt.plot(alp_space,results, label='MAP variability')
plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()

plt.savefig('userwise.png')

In [ ]:

recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_all)
# recommender1.fit(**models_to_combine_best['SLIMER'])
# recommender1.save_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')
recommender1.load_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')

recommender2 = SLIM_BPR_Cython(URM_all)
# recommender2.fit(**models_to_combine_best['SLIMBPR'])
# recommender2.save_model(ofp + 'SLIMBPR_ALL/', 'SLIMBPR_ALL')
recommender2.load_model(ofp + 'SLIMBPR_ALL/', 'SLIMBPR_ALL')

recommenderHybrid = ItemKNNScoresHybridTwoRecommender(URM_all, recommender1, recommender2)
recommenderHybrid.fit(alpha = 0.58)

recommender = recommenderHybrid

import pandas as pd
at = 10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
sep=",",
dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, cutoff=at)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


	# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
	# submission

from datetime import datetime

now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)



In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
CFrecommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
CFrecommender.load_model('../result_experiments/BEST_SLIM_PURECF/', file_name = CFrecommender.RECOMMENDER_NAME + "_my_own_save.zip")

from Recommenders.FeatureWeighting.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg
FW_recommender = CFW_D_Similarity_Linalg(URM_train, ICM_stacked, CFrecommender.W_sparse)
FW_recommender.load_model('../result_experiments/BEST_FW/', file_name = FW_recommender.RECOMMENDER_NAME + "_my_own_save.zip")

In [ ]:
import scipy.sparse as sps
ICM_length_all_5km = ld.getICMlength('5km')
ICM_stacked = sps.hstack((ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km))

In [ ]:
import pandas as pd
argsort_features = np.argsort(-FW_recommender.D_best)

selection_quota = 0.1
        
n_to_select = int(selection_quota*len(argsort_features))
selected_features = argsort_features[:n_to_select]
        
ICM_selected = ICM_stacked.todense()[:,selected_features]
pd.DataFrame(ICM_selected).to_csv('ICM_selected_1.csv', index=False, header=True)


In [ ]:
import scipy.sparse as sps
import pandas as pd
URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, _ = ld.getCOOs()
ICM_length_all_5km = ld.getICMlength('5km')
pd.DataFrame(sps.hstack([ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km]).todense()).to_csv('ICM_ALL.csv', index=False, header=True)

In [ ]:
recommender1 = recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
rank_models(evaluator_validation, [recommender1])
recommender1.fit(**models_to_combine_best['SLIMER'])
rank_models(evaluator_validation, [recommender1])

# recommender1.save_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')
# recommender1.load_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')